**step 2:** prediction model building 

The following imports are done because all previous functions were copied to python script files for easier navigation between notebooks, in this second step we will focus on using the data we cleaned to model predictions of the 'risk_score' parameter.

In [1]:
from src.pipeline import pipeline
from src.preprocess import BeforePipeline
from src.map import data_mapping
from src.feature_engineering import add_engineered_features
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, learning_curve, ShuffleSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.utils import resample
import joblib
import pandas as pd
import numpy as np
import eli5
from eli5.sklearn import PermutationImportance
from src.config import cols_to_drop

38


In [2]:
data_path = '../data/raw/data_chunck.parquet'

In [5]:
def all_data_prep(n): #a function to quickly get how much data we need 
    df = pd.read_parquet(data_path).sample(n=n, random_state=2)
    print(f'Shape before data prep: {df.shape}')

    bp = BeforePipeline()
    df = add_engineered_features(df)

    df = bp.all_before_pipeline(df)
    
    df_y = np.log1p(df['risk_score']) #we log transform y to help the model
    df = df.drop(columns='risk_score')

    print(f'shape after data prep: {df.shape}')
    return df,df_y

### Feature Engineering
We will try to make new features that help the model generalize better by using existing ones.
here are all the features we will add:

Variants of debt to in income ratio:
- revol_bal_to_income = revol_bal / annual_inc
- loan_to_income = loan_amnt / annual_inc
- bc_limit_to_income = total_bc_limit / annual_inc

Credit utilization refinements:
- total_utilization = revol_bal / total_rev_hi_lim
- bc_to_total_limit_ratio = total_bc_limit / total_rev_hi_lim

New Metrics:
- tot_cur_bal_to_income = tot_cur_bal / annual_inc

Credit history features:
- recent_account_ratio = acc_open_past_24mths / total_acc

Risk Buckets & Flags:
- high_util_flag = (revol_util > 80).astype(int)
- short_history_flag = (credit_history_length < 24).astype(int)
- many_inquiries_flag = (inq_last_6mths >= 3).astype(int)


## Introduction to model building
Let's get started with model building, as we are dealing with very high dimensionality data, a lot of non linearity and mixed-type features, the optimal choice for us will probably be a tree based model, We will use a Random Forest Regressor, which is an application of bagging to decision trees with the additional “random subspace” trick: we create a large number of trees and each tree is trained on a bootstrap sample of the data chosen randomly with replacement, and at each split only a portion of the features are randomly chosen without replacement, generally $\sqrt{n}$ features for classification or $n/3$ for regression for $n$ features. 

This will save a ton of computing power while reducing the variance of our model.

In fact, if each tree result is a random variable, let's say we have $n$ random variables ${X_i}$ equally distributed but not necessarly independant, our model output $\tilde{X}$ would then be the average of all the results of the different trees:
$\tilde{X} = \frac{1}{n}\sum_{i=1}^{n} {X_i}$ if we let $Var({X_i}) = \sigma^{2}$ for all $i$ and $Cov({X_i},{X_j}) = \rho \sigma^{2}$ for all $i \neq j$ ($\rho$ being the coeificient of correlation),

Then, by a simple variance calculation we have that, $$Var(\tilde{X}) = \rho \sigma^{2} + \frac{1 - \rho}{n}\sigma^{2}$$
As n grow larger, $\frac{1 - \rho}{n}\sigma^{2} \xrightarrow[n \to \infty]{} 0$, we are then left with $Var(\tilde{X}) = \rho \sigma^{2} \leq  \sigma^{2}$ as $\rho \leq 1$.

So, the more trees we have, the more we reduce our model's output variance, this variance reduction is crucial for improving model stability, especially when evaluating the model across different economic scenarios to assess its robustness.

## Additional Note:
- In classification, the Random Forest aggregates by majority voting (which is equivalent to averaging the 0/1 votes and then thresholding) or by averaging the class probabilities. The variance formula applies to the probability estimates, which can then be thresholded to get the class label. This variance reduction in the probability estimates leads to more stable predictions.

In [6]:
#data collection
X,y = all_data_prep(n=50_000)

print("Infinite values after cleaning:", np.isinf(X.select_dtypes(include=[np.number])).sum().sum())
print("NaN values after cleaning:", X.isnull().sum().sum())
#baseline scikit-learns random forest
rf = RandomForestRegressor(random_state=1,n_jobs=-1)
pipe = pipeline(model=rf, ver=False)


print('total_pymnt_inv' in X.columns)
#cross validation

scores = cross_val_score(estimator=pipe, X=X, y=y, cv=3, scoring='r2')
print(f'cross validation scores: {scores}\nmean: {scores.mean()}')


Shape before data prep: (50000, 148)
===> clean infinites called
===> data_mapping called


d:\pythonProjects\loan_default_prediction\app\src\map.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['risk_score']= data['loan_status'].map(mapping)


===> num_data_prep called
number of cols to drop: 38
data shape before dropping: (50000, 145)
Columns to drop present in data: 28
Columns NOT found in data (cannot drop): ['next_pymnt_d', 'settlement_status', 'hardship_type', 'hardship_loan_status', 'hardship_status', 'hardship_reason', 'debt_settlement_flag_Y', 'debt_settlement_flag_N', 'verification_status_joint', 'sec_app_earliest_cr_line']
28 columns dropped successfully
data shape after dropping: (50000, 117)
===> drop_useless called
===> clean infinites called
shape after data prep: (50000, 116)
Infinite values after cleaning: 0
NaN values after cleaning: 3274
False
[ColumnTransformer] ......... (1 of 5) Processing scale, total=   0.1s
[ColumnTransformer]  (2 of 5) Processing frequency_cols, total=   0.0s
[ColumnTransformer]  (3 of 5) Processing employement_lenght, total=   0.0s
[ColumnTransformer] ... (4 of 5) Processing account_age, total=   0.0s
[ColumnTransformer] ........... (5 of 5) Processing OHE, total=   0.0s
[Pipeline] 

In [9]:
X_train, X_val, y_train, y_val = train_test_split(X,y)

pipe.fit(X_train, y_train)
print('model fitted')
X_trans = pipe['features'].transform(X_val)
print('transformed')
perms = PermutationImportance(pipe[1], scoring='r2', n_iter=3, random_state=1).fit(X_trans, y_val)

[ColumnTransformer] ......... (1 of 5) Processing scale, total=   0.1s
[ColumnTransformer]  (2 of 5) Processing frequency_cols, total=   0.0s
[ColumnTransformer]  (3 of 5) Processing employement_lenght, total=   0.0s
[ColumnTransformer] ... (4 of 5) Processing account_age, total=   0.0s
[ColumnTransformer] ........... (5 of 5) Processing OHE, total=   0.0s
[Pipeline] .......... (step 1 of 2) Processing features, total=   0.2s
[Pipeline] ............. (step 2 of 2) Processing model, total=  46.5s
model fitted
transformed


In [10]:
ohe_cols = pipe[0]['OHE'].get_feature_names_out().tolist()
num_cols = [col for col in X_val.columns if X_val[col].dtype != 'object']

feature_names = num_cols + ['purpose','addr_state'] + ['emp_length'] + ['earliest_cr_line'] + ohe_cols

eli5.show_weights(perms, feature_names=feature_names)

Weight,Feature
0.0695 ± 0.0019,settlement_percentage
0.0408 ± 0.0032,settlement_term
0.0230 ± 0.0007,loan_to_income
0.0167 ± 0.0024,term_ 60 months
0.0079 ± 0.0003,recent_account_ratio
0.0078 ± 0.0032,dti
0.0073 ± 0.0007,max_bal_bc
0.0072 ± 0.0020,all_util
0.0065 ± 0.0007,acc_open_past_24mths
0.0063 ± 0.0010,installment


In [11]:
scores = cross_val_score(estimator=pipe, X=X, y=y, cv=3, scoring='r2')
print(f'cross validation scores: {scores}\nmean: {scores.mean()}')

[ColumnTransformer] ......... (1 of 5) Processing scale, total=   0.1s
[ColumnTransformer]  (2 of 5) Processing frequency_cols, total=   0.0s
[ColumnTransformer]  (3 of 5) Processing employement_lenght, total=   0.0s
[ColumnTransformer] ... (4 of 5) Processing account_age, total=   0.0s
[ColumnTransformer] ........... (5 of 5) Processing OHE, total=   0.0s
[Pipeline] .......... (step 1 of 2) Processing features, total=   0.2s
[Pipeline] ............. (step 2 of 2) Processing model, total=  40.2s
[ColumnTransformer] ......... (1 of 5) Processing scale, total=   0.1s
[ColumnTransformer]  (2 of 5) Processing frequency_cols, total=   0.0s
[ColumnTransformer]  (3 of 5) Processing employement_lenght, total=   0.0s
[ColumnTransformer] ... (4 of 5) Processing account_age, total=   0.0s
[ColumnTransformer] ........... (5 of 5) Processing OHE, total=   0.0s
[Pipeline] .......... (step 1 of 2) Processing features, total=   0.2s
[Pipeline] ............. (step 2 of 2) Processing model, total=  43.9

Model performance are very poor from the get go, let's try some other baseline algorithms that may be more adapted to this task,

Gradient Boosting:

In [ ]:
from xgboost import XGBRegressor

boosted_trees = XGBRegressor()
boosted_pipe = pipeline(boosted_trees, ver=False)
print('Gradient boosting')
scores = cross_val_score(estimator=boosted_pipe, X=X, y=y, cv=3, scoring='r2')
print(f'cross validation scores: {scores}\nmean: {scores.mean()}')

cross validation scores: [0.09892759 0.10311753 0.12699863]
mean: 0.10968125315390416


Support Vector Machines:

In [16]:
from sklearn.svm import NuSVR, SVR

nu_support = NuSVR()
epsilon_support = SVR()

nu_pipe = pipeline(nu_support, ver=False)
ep_pipe = pipeline(epsilon_support, ver=False)

print('Nu Support Vector Regression')
scores = cross_val_score(estimator=nu_pipe, X=X, y=y, cv=3, scoring='r2')
print(f'cross validation scores: {scores}\nmean: {scores.mean()}')
print('\nEpsilon Support Vector Regression')
scores = cross_val_score(estimator=ep_pipe, X=X, y=y, cv=3, scoring='r2')
print(f'cross validation scores: {scores}\nmean: {scores.mean()}')

Nu Support Vector Regression
cross validation scores: [-0.14695707 -0.14982858 -0.14824831]
mean: -0.14834465253542004

Epsilon Support Vector Regression
cross validation scores: [-0.0376405  -0.03907508 -0.0387396 ]
mean: -0.03848506282091672


Neural Networks:

In [17]:
from sklearn.neural_network import BernoulliRBM, MLPRegressor

rbm = BernoulliRBM()
mlp = MLPRegressor()

rbm_pipe = pipeline(rbm, ver=False)
mlp_pipe = pipeline(mlp,ver=False)

print('Bernoulli Restricted Boltzmann Machine')
scores = cross_val_score(estimator=rbm_pipe, X=X, y=y, cv=3, scoring='r2')
print(f'cross validation scores: {scores}\nmean: {scores.mean()}')
print('\nMulti-layer Perceptron regressor')
scores = cross_val_score(estimator=mlp_pipe, X=X, y=y, cv=3, scoring='r2')
print(f'cross validation scores: {scores}\nmean: {scores.mean()}')

Bernoulli Restricted Boltzmann Machine


C:\Users\PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\model_selection\_validation.py:971: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_scorer.py", line 152, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "C:\Users\PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_scorer.py", line 399, in _score
    response_method = _check_response_method(estimator, self._response_method)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\PC\AppData\Local\Packages\PythonSoftwareFoundatio

cross validation scores: [nan nan nan]
mean: nan

Multi-layer Perceptron regressor
cross validation scores: [-37682.21431035   -915.62024768  -4311.85005722]
mean: -14303.228205080932


Stacking weak models:

In [19]:
from sklearn.ensemble import StackingRegressor


estimators = [('rf', rf),('xgb', boosted_trees)]
ensemble_model = StackingRegressor(estimators=estimators, final_estimator=epsilon_support, n_jobs=-1)
ensemble_pipe = pipeline(ensemble_model)


scores = cross_val_score(estimator=ensemble_pipe, X=X, y=y, cv=3, scoring='r2')
print(f'cross validation scores: {scores}\nmean: {scores.mean()}')

[ColumnTransformer] ......... (1 of 5) Processing scale, total=   0.1s
[ColumnTransformer]  (2 of 5) Processing frequency_cols, total=   0.0s
[ColumnTransformer]  (3 of 5) Processing employement_lenght, total=   0.0s
[ColumnTransformer] ... (4 of 5) Processing account_age, total=   0.0s
[ColumnTransformer] ........... (5 of 5) Processing OHE, total=   0.0s
[Pipeline] .......... (step 1 of 2) Processing features, total=   0.2s


KeyboardInterrupt: 